# Loss Run info extraction

In [1]:
import lossrun
import lossrun_models
import matplotlib.pyplot as plt
import cv2

2020-10-28 14:55:24,568 INFO sqlalchemy.engine.base.Engine select version()
2020-10-28 14:55:24,569 INFO sqlalchemy.engine.base.Engine {}
2020-10-28 14:55:24,572 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-10-28 14:55:24,573 INFO sqlalchemy.engine.base.Engine {}
2020-10-28 14:55:24,576 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-28 14:55:24,577 INFO sqlalchemy.engine.base.Engine {}
2020-10-28 14:55:24,579 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-28 14:55:24,580 INFO sqlalchemy.engine.base.Engine {}
2020-10-28 14:55:24,581 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-10-28 14:55:24,582 INFO sqlalchemy.engine.base.Engine {}
2020-10-28 14:55:24,585 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [13]:
lossrun.transform_to_images_an_entire_folder('/home/zned897/Proyects/pdf_text_extractor/pdfTextanalyzer/TEMPORAL/Demo/pdfs/', '/home/zned897/Proyects/pdf_text_extractor/pdfTextanalyzer/TEMPORAL/Demo/images/', format='.png' )

lossrun.transform_to_text_an_entire_folder('/home/zned897/Proyects/pdf_text_extractor/pdfTextanalyzer/TEMPORAL/Demo/images/', '/home/zned897/Proyects/pdf_text_extractor/pdfTextanalyzer/TEMPORAL/Demo/text/')

True

## Load data and clasify as loss report, npdb and emails

In [2]:
## load file as image and text
_file = 'AI NPDB'
_image_file = 'images/'+ _file + '.png'
_txt_file = 'text/'+ _file + '.txt'

# load image
_image = cv2.imread(_image_file, cv2.IMREAD_GRAYSCALE)
# load text
txt_dict = lossrun.read_dict(_txt_file)

# check the result
print('The file is a:', lossrun.is_report(_image, txt_dict)[0])
plt.figure(figsize=(30,25)),plt.imshow(_image, cmap = 'gray'),plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'text/AI NPDB.txt'

## Process acording the report

In [57]:
# Check the report type and determinate the rules accordig to file type 
report_type, topic_conf, ent_conf = lossrun.is_report(_image,txt_dict)
suspects = lossrun.search_rules(txt_dict, topic_conf)


## Relate the information acording text distribution and NER model

In [58]:
import spacy 
from configobj import ConfigObj

if (report_type == 'NPDB') or (report_type == 'EMAIL'):
    print('loading NPDB NER model...')
    nlp = spacy.load('./config/NPDB_ner_model')
    Topics = ConfigObj('./config/config_npdb_entites.ino')
else:
    print('loading lossrun NER model...')
    nlp = spacy.load('./config/lossrun_ner_model')
    Topics = ConfigObj('./config/config_lossrun_entites.ino') 

loading NPDB NER model...


In [59]:
import re
regexp_act = r"\b[A-Z][A-Z]+\b"
spatial_filter = lossrun.spatial_filter(txt_dict, suspects, report_type)
spatial_filter_topics = len(spatial_filter)

for i in range (len(spatial_filter)):
    sentence  = ' '.join(spatial_filter[i])
    sentence = re.sub('\s+',' ', sentence)
    doc = nlp(sentence)
    #print(suspects[i][0],sentence)
    for ent in doc.ents:
        print(ent.text, ent.label_)
        pass

MCKAY, JAMES WOODROW III ORG
JULIAN F. KEITH ADATC ORG
06/23/2017 DATE
JULIAN F. KEITH ADATC ORG


In [60]:
for i in range (len(spatial_filter)):
    sentence  = ' '.join(spatial_filter[i])
    sentence = re.sub('\s+',' ', sentence)
    print(sentence)

Name: MCKAY, JAMES WOODROW III
Name: JULIAN F. KEITH ADATC
Date: 06/23/2017 Page:1 of 1
Name: JULIAN F. KEITH ADATC (DBID ending in ...45)
Reports Found Based on the Subject Information


In [61]:
from datetime import datetime
pract_name, ref_number, ent_name, paid_by, total_pract, outcome, init_act , act_basis = [],[],[],[],[],[],[],[]

proc_date = [datetime(1900,1,1)]
event_date = [datetime(1900,1,1)]
paid_date = [datetime(1900,1,1)]
relevant = True

for topic in range(spatial_filter_topics):
    #clean the sentece
    #spatial_filter[topic] = list(dict.fromkeys(spatial_filter[topic]))
    sentence  = ' '.join(spatial_filter[topic])
    sentence = re.sub('\s+',' ', sentence)
    doc = nlp(sentence)
    #print(sentence)
    for ent in doc.ents:
        if suspects[topic][0] == 'pract_name':
            if ent.label_ in Topics['pract_name']:
                pract_name.append(ent.text)
        if suspects[topic][0] == 'ref_number':
             if ent.label_ in Topics['ref_number']:
                ref_number.append(ent.text)
        if suspects[topic][0] == 'proc_date':
            if ent.label_ in Topics['proc_date']:
                try:
                    proc_date.append(datetime.strptime(ent.text, '%m/%d/%Y'))
                except:
                    pass
        if suspects[topic][0] == 'paid_date':
            if ent.label_ in Topics['paid_date']:
                try:
                    paid_date.append(datetime.strptime(ent.text, '%m/%d/%Y'))
                except:
                    pass
        if suspects[topic][0] == 'event_date':
            if ent.label_ in Topics['event_date']:
                try:
                    event_date.append(datetime.strptime(ent.text, '%m/%d/%Y'))
                except:
                    pass
        if suspects[topic][0] == 'ent_name':
            if ent.label_ in Topics['ent_name']:
                ent_name.append(ent.text)
        if suspects[topic][0] == 'paid_by':
            if ent.label_ in Topics['ent_name']:
                paid_by.append(ent.text)
        if suspects[topic][0] == 'total_pract':
            if ent.label_ in Topics['total_pract']:
                total_pract.append(ent.text)
    
    if suspects[topic][0] == 'outcome':
        outcome.append(sentence)
    if suspects[topic][0] == 'init_act':   
        aux = re.findall(regexp_act, sentence)
        sentence = ' '.join(aux)
        init_act.append(sentence) if sentence is not '' else None
    if suspects[topic][0] == 'relevant':
        relevant = False

pract_name = str(pract_name)[1:-1]
ref_number = str(ref_number)[1:-1]
#proc_date = str(proc_date)[1:-1]
#paid_date =  str(paid_date)[1:-1]
#event_date = str(event_date)[1:-1]
ent_name =  str(ent_name)[1:-1]
paid_by = str(paid_by)[1:-1]
total_pract = str(total_pract)[1:-1]+'0'
total_pract = int(float(re.sub("[$|,|']", '',total_pract)))
outcome = str(outcome)[1:-1]
init_act = str(init_act)[1:-1]


In [62]:
total_pract

0

In [63]:
print('pract_name:' + pract_name)
#print('ref_number:' + str(ref_number))
print('proc_date:' + str(proc_date))
print('paid_date:' + str(paid_date))
print('event_date:' + str(event_date))
print('paid_by:' + str(paid_by))
print('total_pract:' + str(total_pract))
print('outcome:' + str(outcome))
print('ent_name:' + str(ent_name))
print('init_act:' + str(init_act))
print('relevance:' + str(relevant))


pract_name:'MCKAY, JAMES WOODROW III', 'JULIAN F. KEITH ADATC'
proc_date:[datetime.datetime(1900, 1, 1, 0, 0), datetime.datetime(2017, 6, 23, 0, 0)]
paid_date:[datetime.datetime(1900, 1, 1, 0, 0)]
event_date:[datetime.datetime(1900, 1, 1, 0, 0)]
paid_by:
total_pract:0
outcome:
ent_name:'JULIAN F. KEITH ADATC'
init_act:
relevance:False


In [64]:
import datetime
if report_type == 'NPDB':
    lossrun_models.npdbRecord(process_date = max(proc_date),
                          practitioner_name = pract_name,
                          action_initial = init_act,                  
                          action_basis = act_basis, 
                          entity_name = ent_name,
                          payment_date = max(paid_date), 
                          payment_total_amount = total_pract,
                          event_day = max(event_date), 
                          event_outcome = outcome,
                          event_paid_by = '')
elif report_type == 'lossrun':
    pass


2020-10-28 12:29:40,467 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-28 12:29:40,469 INFO sqlalchemy.engine.base.Engine INSERT INTO event_npdb (event_day, event_outcome, event_paid_by) VALUES (%(event_day)s, %(event_outcome)s, %(event_paid_by)s) RETURNING event_npdb.event_id
2020-10-28 12:29:40,470 INFO sqlalchemy.engine.base.Engine {'event_day': datetime.datetime(1900, 1, 1, 0, 0), 'event_outcome': '', 'event_paid_by': ''}
2020-10-28 12:29:40,472 INFO sqlalchemy.engine.base.Engine INSERT INTO payment_npdb (payment_date, payment_total_amount) VALUES (%(payment_date)s, %(payment_total_amount)s) RETURNING payment_npdb.payment_id
2020-10-28 12:29:40,473 INFO sqlalchemy.engine.base.Engine {'payment_date': datetime.datetime(1900, 1, 1, 0, 0), 'payment_total_amount': 0}
2020-10-28 12:29:40,475 INFO sqlalchemy.engine.base.Engine INSERT INTO action_npdb (action_initial, action_basis) VALUES (%(action_initial)s, %(action_basis)s) RETURNING action_npdb.action_id
2020-10-28 12:29:4

# Auxiliar code.
## Ignore it.


## Relate the information acording named enity recognition


In [65]:
topic = 19
sent = ' '.join(spatial_filter[topic])
doc = nlp(' '.join(spatial_filter[topic]))
print(sent)

for ent in doc.ents:
    print(ent.text, ent.label_)


IndexError: list index out of range

In [13]:
import string as String
printable = set(String.printable)

for i in range(len(spatial_filter)):

    string = ' '.join(spatial_filter[:][i])
    string = re.sub('\s+',' ',string)
    
    
    # remove non printalbes elemts
    string = ''.join(filter(lambda x: x in printable, string))
    #print (string)
    doc = nlp(string)

    for ent in doc.ents:
        print('It will be added to data base : ')
        print(ent.text + ' in ' + ent.label_)
        print('\n.................')

It will be added to data base : 
BORIS, GEORGE THEO in PERSON

.................
It will be added to data base : 
05/22/2018 in DATE

.................
It will be added to data base : 
05/22/2018 in DATE

.................
It will be added to data base : 
05/22/2018 in DATE

.................
It will be added to data base : 
11/15/2016 in DATE

.................
It will be added to data base : 
11/15/2016 in DATE

.................
It will be added to data base : 
11/15/2016 in DATE

.................
It will be added to data base : 
11/15/2016 in DATE

.................
It will be added to data base : 
11/15/2016 in DATE

.................
It will be added to data base : 
11/01/2016 in DATE

.................
It will be added to data base : 
02/16/2009 in DATE

.................
It will be added to data base : 
ADMIRAL INSURANCE COMPANY in ORG

.................
It will be added to data base : 
$ 87,500.00 in MONEY

.................
It will be added to data base : 
- SETTLEMENT in MO

## Relate the information acording contextual relation

In [14]:
# load contextual model
import string as String
printable = set(String.printable)

for i in range(len(spatial_filter)):

    string = ' '.join(spatial_filter[:][i])
    string = re.sub('\s+',' ',string)
    
    
    # remove non printalbes elemts
    string = ''.join(filter(lambda x: x in printable, string))
    #print (string)
    _suspect = nlp(string)
    _topic = nlp(suspects[i][0])
    print("similar: " + string, _suspect.similarity(_topic) , i)

similar: Name: BORIS, GEORGE THEO 0.0 0
similar: Name: 0.0 1
similar: Number: N57550213 0.0 2
similar: Date: 05/22/2018 0.0 3
similar: Date: 05/22/2018 NATIONAL PRACTITIONER DATA  0.0 4
similar: Date: 05/22/2018 NATIONAL PRACTITIONER DATA BANK NPDB 0.0 5
similar: Date: 11/15/2016 NATIONAL PRACTITIONER DATA BANK  0.0 6
similar: Date: 11/15/2016 NATIONAL PRACTITIONER DATA BANK  0.0 7
similar: Date: 11/15/2016 NATIONAL PRACTITIONER DATA BANK NPDB 0.0 8
similar: Date: 11/15/2016 NATIONAL PRACTITIONER DATA BANK NPDB 0.0 9
similar: Date: 11/15/2016 0.0 10
similar: of This Payment: 11/01/2016 0.0 11
similar: of Event Associated With Allegation or Incident: 02/16/2009 0.0 12
similar: Name: ADMIRAL INSURANCE COMPANY 0.0 13
similar: Name 0.0 14
similar: Name 0.0 15
similar: Name 0.0 16
similar: Name 0.0 17
similar: Name 0.0 18
similar: Name 0.0 19
similar: to Be Paid by 0.0 20
similar: to Be Paid by This Payer for All 0.0 21
similar: PAYER FOR THIS PRACTITIONER 0.0 22
similar: Payer for This Pra

## Insert candidates to data base

In [15]:
#Lossrun_models.registerRecord(timeDimDay = 12,
 #                             timeDimMonth = 9,
  #                            timeDimYear = 20,
   #                           policyDimStatus = "Open", 
    #                          reportGeneratorDimName = _temp[encuentra(_temp,'ORG')][0], 
     #                         insuredDimName = _temp[encuentra(_temp,'ORG')][0],
      #                        insurerDimName = _temp[encuentra(_temp,'PERSON')][0], 
       #                       statusName = "Open", 
        #                      lossRunReportDimDate = datetime.datetime.now()) 

### Contextual data
